




**Data Parser for grabbing clinical Trial eligibility criteria**

In [ ]:
# CS 333 Final Project Data Parser


import requests
import json

from google.colab import drive
drive.mount('/content/drive')

# api_url = 'https://clinicaltrials.gov/api/query/full_studies?expr=heart+attack&aggFilters=results:with,status:com&min_rnk=1&max_rnk=2&fmt=json'

# single study
# https://clinicaltrials.gov/study/NCT05940077?rank=1&viewType=Table&aggFilters=results:with,status:com&tab=results
api_url = 'https://clinicaltrials.gov/api/v2/studies?aggFilters=results:with,status:com'


params = {
}

response = requests.get(api_url)

if response.status_code == 200:
    print(response.status_code)
else:
    print("Error: ", response.status_code, response.text)

with open("/content/drive/My Drive/outfile_test.json", "w") as outfile:
    raw_json = response.json()
    json_object = json.dumps(raw_json, indent=4)

    # study_data = {}

    # study_data['identificationModule'] = raw_json['studies'][0]['protocolSection']['identificationModule']

    # study_data['eligibilityModule'] = raw_json['studies'][0]['protocolSection']['eligibilityModule']

    # all_data = json.dumps(study_data, indent=4)

    # outfile.write(all_data)

    outfile.write(json_object) # (testing)

    print("Wrote successfully")





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
200
Wrote successfully


In [ ]:
api_url = 'https://clinicaltrials.gov/api/v2/studies?aggFilters=results:with,status:com'

params = {
}

response = requests.get(api_url)

if response.status_code == 200:
    print(response.status_code)
else:
    print("Error: ", response.status_code, response.text)

with open("/content/drive/My Drive/outfile.json", "w") as outfile:
    raw_json = response.json()
    json_object = json.dumps(raw_json, indent=4)

    study_keys = {'studies': []}

    for study in raw_json['studies']:
      study_data = {}

      study_data['identificationModule'] = study['protocolSection']['identificationModule']

      study_data['eligibilityModule'] = study['protocolSection']['eligibilityModule']

      study_keys['studies'].append(study_data)

    outfile.write(json.dumps(study_keys, indent=4))

    # outfile.write(json_object) # (testing)

    print("Wrote successfully")

200
Wrote successfully


**Code for performing named entity recognition on very simple labeled patient health records**

In [41]:
import json
import spacy

from google.colab import drive
drive.mount('/content/drive')

with open("/content/drive/My Drive/consolidated_chia_outfile.json", "r") as outfile:
  data = json.load(outfile)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!python -m spacy download en_core_web_lg

2023-11-20 01:18:25.049429: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 01:18:25.049489: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 01:18:25.049519: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 01:18:26.275950: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
data["NCT00894712_inc"]["Annotations"][0]

'Procedure 10 24--*--pathologically'

In [5]:
data["NCT00894712_inc"]["Text"]

'Must have pathologically confirmed invasive adenocarcinoma or ductal carcinoma in situ of the breast. . Patients must have undergone segmental mastectomy (i.e., lumpectomy). . Patients must not have received prior radiation therapy to the breast. . Patients must not have active local-regional disease prior to registration. . Patients must not be pregnant because of the potential for fetal harm as a result of radiation treatment. Women of child-bearing age will be given a serum pregnancy test prior to study entry to ensure they are not pregnant. They will also be counseled on the importance of avoiding pregnancy and hormonal contraception while undergoing radiation therapy. . Patients must not have a serious medical or psychiatric illness which prevents informed consent or compliance with treatment. . All patients must be informed of the investigational nature of this study and give written informed consent in accordance with institutional and federal guidelines. . '

In [6]:
from spacy.tokens import DocBin

# nlp = spacy.blank("en")

nlp = spacy.load("en_core_web_lg")

doc_bin = DocBin()

In [45]:
# training from chia data
from spacy.util import filter_spans

training_data = []

for example in data:

  if "Text" not in data[example] and "Annotations" not in data[example]:
    continue

  temp_lst = []

  text = data[example]["Text"]
  text = text.replace(".", " ")
  text = text.strip()

  if text == '.' or text == '' or text == ',' or text == ':':
    continue;

  labels = []
  for entity in data[example]["Annotations"]:
    start = entity.split('--*--')[0].split(' ')[1]
    end = entity.split('--*--')[0].split(' ')[2]
    label = entity.split('--*--')[0].split(' ')[0]
    labels.append((start, end, label))

  doc = nlp.make_doc(text)
  ents = []

  for start, end, label in labels:
    if start.isdigit() and end.isdigit():
      span = doc.char_span(int(start), int(end), label=label, alignment_mode="contract")
      print(span)
      if span is None:
        print("Skipping entity")
      else:
        ents.append(span)
  filtered_ents = filter_spans(ents)
  doc.ents = filtered_ents
  doc_bin.add(doc)

doc_bin.to_disk("train.spacy")


Streaming output truncated to the last 5000 lines.
infection, hepatic encephalopathy, hepatorenal syndrome, gastrointestinal bleeding
immunomodulatory therapy
antineoplastic or immunomodulatory therapy
past 12 months
IFN
anti hepatitis B virus
resistance
NAs drug
Patients who can't come back to clinic for follow-up on schedule
Osteonecrosis
decompression surgery
planned
autologous stem cell transplant
Unable to participate
administrative reasons
Psychiatric troubles
Pain at rest
critical limb ischemia
Unable to walk
wheelchair subjects
PD
by a neurologist
by a neurologist
None
Skipping entity
30 to 85
modified Hoehn and Yahr (
) stage 1 5 to
  able and willing to give written consent for participation in the
  able and willing to give written consent for participation in the
;   living at home in the
for 30
Ulcerative colitis
moderate to severe
clinical remission
by the first course of corticosteroids
first course of corticosteroids
first course
corticosteroids
without
steroid
during l

TypeError: ignored

In [ ]:
# training from artificial data (LOCATION, AGE, DISEASE)
from spacy.util import filter_spans

training_data = []

for example in data["data"]:
  temp_lst = []

  text = example['entities'][0]['text']

  labels = []
  for entity in example['entities']:
    start = entity['indexes'].split(',')[0]
    end = entity['indexes'].split(',')[1]
    label = entity['label']
    labels.append((start, end, label))

  doc = nlp.make_doc(text)
  ents = []

  for start, end, label in labels:
    span = doc.char_span(int(start), int(end), label=label, alignment_mode="contract")
    if span is None:
      print("Skipping entity")
    else:
      ents.append(span)
  filtered_ents = filter_spans(ents)
  doc.ents = filtered_ents
  doc_bin.add(doc)

doc_bin.to_disk("train.spacy")


In [52]:
for doc in doc_bin.get_docs(nlp.vocab):
    # Now you can work with each Doc object
    print(doc.cats)

Streaming output truncated to the last 5000 lines.
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{

KeyboardInterrupt: ignored

In [46]:
!python -m spacy init fill-config /content/drive/My\ Drive/base_config.cfg config.cfg

2023-11-20 02:27:30.310889: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 02:27:30.310951: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 02:27:30.310993: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 02:27:32.146770: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy init fill-config --help

2023-11-14 16:30:27.061787: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 16:30:27.061856: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 16:30:27.061891: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 16:30:28.220072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
                                                                                                    
 Usage: python -m spacy init fill-config [OPTIONS] BASE_PATH [OUTPUT_FILE]                          
                                    

In [47]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

2023-11-20 02:27:43.377907: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 02:27:43.377973: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 02:27:43.378008: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 02:27:44.633389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    retu

In [16]:
nlp_ner = spacy.load("model-best")

In [17]:
doc = nlp_ner("Patient has severe leg pain. Patient is taking dosages of immunoglobulins. Patient is undergoing sling surgery.")

colors = {"Condition": "#F67DE3", "Drug": "#7DF6D9", "Procedure":"#a6e22d"}
options = {"colors": colors}
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

# print(doc.ents)



/usr/local/lib/python3.10/dist-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


()
